In [10]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec


In [12]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '2afcea0378d96a4b3d8811034c821631', 'Date': 'Sun, 25 Aug 2024 07:28:57 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [13]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Alice Johnson',
  'subject': 'Introduction to Computer Science',
  'stars': 5,
  'review': 'Excellent professor, very clear explanations and approachable.'},
 {'professor': 'Dr. Bob Smith',
  'subject': 'Calculus I',
  'stars': 4,
  'review': 'Challenging course, but the professor is very supportive.'},
 {'professor': 'Dr. Carol Lee',
  'subject': 'Physics for Engineers',
  'stars': 3,
  'review': 'The material is difficult, but Dr. Lee makes it manageable.'},
 {'professor': 'Dr. Daniel Kim',
  'subject': 'Organic Chemistry',
  'stars': 2,
  'review': 'Hard to follow lectures, but the lab sessions are helpful.'},
 {'professor': 'Dr. Emily Davis',
  'subject': 'World History',
  'stars': 5,
  'review': 'Amazing storytelling skills, keeps the class engaged.'},
 {'professor': 'Dr. Frank Miller',
  'subject': 'Macroeconomics',
  'stars': 4,
  'review': 'Well-structured course, but exams are tough.'},
 {'professor': 'Dr. Grace Brown',
  'subject': 'English Literature',
 

In [14]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values":embedding,
        "id":review["professor"],
        "metadata":{
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"],
        }
    })

In [15]:
processed_data[0]

{'values': [-0.01927141,
  -0.03043934,
  -0.037440132,
  0.004808236,
  -0.009315156,
  -0.015142738,
  -0.038388956,
  0.02795188,
  -0.039414715,
  -0.013565636,
  0.040158387,
  -0.026041407,
  -0.011950069,
  -0.015758192,
  -0.0015386356,
  0.0066802427,
  0.019707358,
  0.035593767,
  0.01456575,
  0.055698607,
  0.022130707,
  -0.023553945,
  0.010456311,
  0.042235546,
  -0.049056828,
  -0.057955272,
  0.021528075,
  0.018014858,
  -0.020079194,
  -0.00070280384,
  0.07688049,
  -0.013514348,
  -0.019745823,
  -0.042389408,
  -0.034106422,
  0.030285476,
  -0.012988648,
  0.020002263,
  -0.025451597,
  0.025310554,
  0.025002828,
  0.012834785,
  -0.003593355,
  -0.019848399,
  0.021233171,
  0.009090772,
  -0.046389863,
  -0.0011443602,
  0.024733566,
  0.0252208,
  -0.021566542,
  -0.015565863,
  0.062673755,
  0.017219895,
  -0.06877701,
  0.028003167,
  0.004522947,
  0.026567107,
  0.0059237466,
  -0.043133084,
  0.073803216,
  -0.0077701095,
  0.023156464,
  -0.024528414

In [16]:
# Insert the embeddings into the Pinecone index
index = pc.Index("rag")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)

In [17]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}